# Using Regression with CAPM
This code is for making a simple CAPM Regression Model using Dr. French's Historical Data and the TD Ameritrade API's Candle Data

It takes data from: 
- TD Ameritrade and tda-api: https://developer.tdameritrade.com/ & https://tda-api.readthedocs.io/en/latest/
- Dr. Kenneth R. French's Data Library(for risk-free rates and market rates): https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

This notebook also utilizes some prewritten functions to pull data without the big setups required for the tda-api, and can be found in the configs.py file in the functions subfolder

The Configs and CSV Data Files will be saved in the repository /Users/dB/.secret/ under the filename 'tda-api-v6.json' and pathname 'ff-research-data'. Their paths are listed here for easier access while I write in this notebook:

/Users/dB/.secret/tda-api-v6.json

/Users/dB/.secret/ff-research-data/

## Step 1: Import Necessary Packages and Add Configs

In [14]:
# Imports -----------------------------------------------------------------------------
import functions_for_notebooks.configs as c
from tda.client import Client
from tda import auth as a
import pandas as pd
import datetime as dt

# Configs -----------------------------------------------------------------------------
tda_api_json_path = '/Users/dB/.secret/tda-api-v6.json'
tda_api_json = c.get_account_data(tda_api_json_path)

ff_data_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors.CSV'
ff_data_weekly_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors_weekly.CSV'
ff_data_daily_path = '/Users/dB/.secret/ff-research-data/F-F_Research_Data_Factors_daily.csv'

start_datetime = '01/01/2019' # Collect Data from the Last 5 Years

symbol = 'AMD'
periods = '1d'
periods2 = '1w'
end = dt.datetime.now() # Today's Date
start = end - dt.timedelta(days = 1825) # Subtract 5 Years from Today

## Step 2: Collect Data and Process into DataFrames

To collect the data, we need to read Dr. French's CSVs for data and call the TD Ameritrade API for candle data.

Reading Dr. French's CSVs is a simple process, all we need to do is decide to use daily or weekly data and then call Panda's read_from_csv() function. We will need to translate the dates into datetime objects to match up with the API data [ADD LATER]

Calling the API will pull down candle data for a stock. This will provide us with the open, close, high, low and volume data for the specified timeframe. Since Dr. French's Data only has daily and weekly entries for the data, we should only call tda-api's Client.get_price_history_every_day() or Client.get_price_history_every_week() functions so the dates match. 

I have a function from the dBI project named get_price_history_df() that can call the API once a Client object is created, and we can pass in '1d' and '1w' as the periods parameter for the daily and weekly data.

In [13]:
import functions_for_notebooks.configs as c
# Get the Data from the CSV
daily_data = c.read_ff_csvs(ff_data_daily_path)
weekly_data = c.read_ff_csvs(ff_data_weekly_path)

AttributeError: module 'functions_for_notebooks.configs' has no attribute 'read_ff_csvs'

In [ ]:
# Get our API Parameters for the Candle Data Call
ak = tda_api_json['api_key']
ru = tda_api_json['redirect_uri']
tp = tda_api_json['token_path']

# Create the tda.Client object to call the API
client = c.connect_to_api(ak, ru, tp)

# Call the API for candle data
candles_daily = c.get_price_history_df(client, symbol, '1d', start, end)
candles_weekly = c.get_price_history_df(client, symbol, '1w', start, end)